<a href="https://colab.research.google.com/github/shivendr7/SatGAN/blob/master/generator_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from tensorflow.keras.utils import get_file
from tensorflow.io import read_file
from skimage import data, color
from skimage.transform import resize
url='http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/maps.tar.gz'
path_to_zip=get_file('maps.tar.gz', origin=url, extract=True)
inside_train=listdir('/root/.keras/datasets/maps/train')
src_list=[]
full_path='/root/.keras/datasets/maps/train/'
for filename in inside_train:
  pixels=load_img(full_path+filename, target_size=(256,512))
  pixels=img_to_array(pixels)
  src_list.append(pixels[:,:256])
src_list=np.asarray(src_list)
resized_list=[]
for i in src_list:
  image=i
  image_resized = resize(image, (image.shape[0] // 2, image.shape[1] // 2),
                        anti_aliasing=True)
  resized_list.append(image_resized)
resized_list=np.array(resized_list)
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(src_list[0].astype('uint8'))
plt.subplot(1,2,2)
plt.imshow(resized_list[0].astype('uint8'))

In [32]:
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, Concatenate, Lambda, Subtract, Add, Activation, Reshape, Multiply, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from keras.utils.vis_utils import plot_model
from tensorflow.keras.initializers import RandomNormal
import tensorflow as tf
from keras.losses import MeanSquaredError 
import keras.backend as K
import numpy as np

In [2]:
def dense_block(in_layer):
  init=RandomNormal(stddev=0.2)
  conv1=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(in_layer)
  conv11=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv1)
  conv12=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv1)
  conv13=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv1)
  concat1=Concatenate()([conv11, conv12, conv13])
  conv201=Concatenate()([concat1, conv11])
  conv202=Concatenate()([concat1, conv12])
  conv203=Concatenate()([concat1, conv13])
  conv21i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv201)
  conv22i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv202)
  conv23i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv203)
  conv21=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv21i)
  conv22=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv22i)
  conv23=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv23i)
  concat2=Concatenate()([conv21, conv22, conv23])
  conv301=Concatenate()([concat2, conv21i])
  conv302=Concatenate()([concat2, conv22i])
  conv303=Concatenate()([concat2, conv23i])
  conv31i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv301)
  conv32i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv302)
  conv33i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv303)
  conv31=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv31i)
  conv32=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv32i)
  conv33=Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=init)(conv33i)
  concat3=Concatenate()([conv31, conv32, conv33])
  conv401=Concatenate()([concat3, conv31i])
  conv402=Concatenate()([concat3, conv32i])
  conv403=Concatenate()([concat3, conv33i])
  conv41i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv401)
  conv42i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv402)
  conv43i=Conv2D(filters=64, kernel_size=(1,1), padding='same', kernel_initializer=init)(conv403)
  concat4=Concatenate()([conv41i, conv42i, conv43i])
  out_layer=Concatenate()([concat4, conv1])
  return out_layer

In [ ]:
inp=Input(shape=(28,28,1))
m=Model(inputs=inp, outputs=dense_block(inp))
plot_model(m)

In [4]:
def SubpixelConv2D(input_shape, scale=4):
    """
    Keras layer to do subpixel convolution.
    NOTE: Tensorflow backend only. Uses tf.depth_to_space
    Ref:
        [1] Real-Time Single Image and Video Super-Resolution Using an Efficient Sub-Pixel Convolutional Neural Network
            Shi et Al.
            https://arxiv.org/abs/1609.05158
    :param input_shape: tensor shape, (batch, height, width, channel)
    :param scale: upsampling scale. Default=4
    :return:
    """
    # upsample using depth_to_space
    def subpixel_shape(input_shape):
        dims = [input_shape[0],
                input_shape[1] * scale,
                input_shape[2] * scale,
                int(input_shape[3] / (scale ** 2))]
        output_shape = tuple(dims)
        return output_shape

    def subpixel(x):
        return tf.nn.depth_to_space(x, scale)

    return Lambda(subpixel, output_shape=subpixel_shape, name='subpixel')

In [6]:
def build_udsn(image_shape):
  inp=Input(shape=image_shape)
  db1=dense_block(inp)
  con=Concatenate()([db1, db1])
  db2=dense_block(con)
  con=Concatenate()([db1, db2, db2])
  db3=dense_block(con)
  con=Concatenate()([db1, db2, db3, db3])
  db4=dense_block(con)
  con=Concatenate()([db1, db2, db3, db4, db4])
  db5=dense_block(con)
  con=Concatenate()([db1, db2, db3, db4, db5, db5])
  db6=dense_block(con)
  LR=Conv2D(filters=12, kernel_size=(3,3), padding='same')(db6)
  LR=LeakyReLU(alpha=0.2)(LR)
  Ibase=SubpixelConv2D(LR.shape, scale=2)(LR)
  model=Model(inputs=inp, outputs=Ibase)
  return model

In [ ]:
udsn=build_udsn(image_shape=(128,128,3))
plot_model(udsn)

In [ ]:
udsn.summary()

In [9]:
#eesn essentials
def meanfilter(shape, dtype=None):   #kernel to find mean of 3-channel pixel data
  f=np.array([
              [[[.3333],
                [.3333],
                [.33333]]]
  ])
  return K.variable(f, dtype='float32')
def laplacian(shape, dtype=None):   #laplacian kernel
  f=np.array([
      [[[-1]],[[-1]],[[-1]]],
      [[[-1]],[[ 8]],[[-1]]],
      [[[-1]],[[-1]],[[-1]]]
  ])
  return K.variable(f, dtype='float32')
def channel_thrice(shape, dtype=None):   #kernel to triplicate the single channel data
  f=np.array([
              [[[1, 1, 1]]]   #3 for three layers of filters
  ])
  return K.variable(f, dtype='float32')

In [10]:
def build_eesn(Ibase_shape):
  inp=Input(shape=(Ibase_shape))
  bw=Conv2D(filters=1, kernel_size=(1,1), padding='same', kernel_initializer=meanfilter)(inp)
  Iedge=Conv2D(filters=1, kernel_size=(3,3), padding='same', kernel_initializer=laplacian, trainable=False)(bw)     #no change recommended
  Itrip=Conv2D(filters=3, kernel_size=(1,1), padding='same', kernel_initializer=channel_thrice, trainable=False)(Iedge)   #no change recommended
  
  sub=Subtract()([inp, Itrip])
  ee=Conv2D(filters=128, kernel_size=(3,3), padding='same', strides=(2,2))(Iedge)
  ee=Conv2D(filters=256, kernel_size=(3,3), padding='same')(ee)
  ee=Conv2D(filters=64, kernel_size=(1,1), padding='same')(ee)
  
  #dense net
  db=dense_block(ee)
  db=dense_block(db)
  db=dense_block(db)
  db=Conv2D(56, kernel_size=(3,3), padding='same')(db)

  #mask branch
  mb=Conv2D(filters=64, kernel_size=(3,3), padding='same')(ee)
  mb=LeakyReLU(alpha=0.2)(mb)
  mb=Conv2D(filters=128, kernel_size=(3,3), padding='same')(mb)
  mb=LeakyReLU(alpha=0.2)(mb)
  mb=Conv2D(filters=56, kernel_size=(3,3), padding='same')(mb)
  mb=LeakyReLU(alpha=0.2)(mb)
  mb=Activation('sigmoid')(mb)

  ee=Conv2D(4, kernel_size=(3,3), padding='same')(Multiply()([mb, db]))
  Istedge=SubpixelConv2D(ee.shape, scale=2)(ee)
  #Istedge=Conv2D(1, kernel_size=(3,3), padding='same')(Istedge)
  Istedge=Conv2D(filters=3, kernel_size=(1,1), padding='same', kernel_initializer=channel_thrice, trainable=False)(Istedge)  #no change recommended
  SR=Add()([Istedge, sub])
  model=Model(inputs=inp, outputs=SR)
  return model

In [ ]:
eesn=build_eesn(Ibase_shape=(256,256,3))
plot_model(eesn)

In [ ]:
eesn.summary()

In [62]:
def build_discriminator(img_shape):
  def block(Input, k, n, s):
    l1=Conv2D(n, kernel_size=(k,k), strides=(s,s), padding='same')(Input)
    Bn=BatchNormalization()(l1)
    Lrlu=LeakyReLU(alpha=0.2)(Bn)
    return Lrlu
  I1=Input(shape=img_shape)
  c1=Conv2D(64, kernel_size=(3,3), padding='same')(I1)
  l1=LeakyReLU(alpha=0.2)(c1)
  b1=block(l1, 3, 64, 2)
  b2=block(b1, 3, 128, 1)
  b3=block(b2, 3, 128,2)
  b4=block(b3, 3, 256, 1)
  b5=block(b4, 3, 256, 2)
  b6=block(b5, 3, 512, 1)
  b7=block(b6, 3, 512, 2)
  b8=Flatten()(b7)
  b9=Dropout(.5)(b8)
  l8=Dense(1024)(b9)
  b10=Dropout(.25)(l8)
  l9=LeakyReLU(alpha=0.2)(b10)
  l10=Dense(1, activation='sigmoid')(l9)
  m=Model(inputs=I1, outputs=l10)
  opt=Adam(learning_rate=0.002, beta_1=0.5)
  m.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return m

In [ ]:
disc=build_discriminator(img_shape=(256,256,3))
disc.summary()

In [19]:
from keras.applications.vgg16 import VGG16

In [36]:
vgg=VGG16()
vgg.trainable=False
resize_layer=Conv2D(filters=3, kernel_size=(33,33), kernel_initializer=RandomNormal(stddev=0.2), trainable=False)

In [48]:
def charbonnier_penalty(y_true, y_pred):
  epsilon=0.001 #compensation parameter
  return K.mean((y_true-y_pred)**2 + epsilon**2)

def charbonnier_model(img_shape):
  inp=Input(shape=img_shape)
  Ibase=udsn(inp)
  resize_Ibase=resize_layer(Ibase)
  out=vgg(resize_Ibase)
  model=Model(inputs=inp, outputs=out)
  opt=Adam(learning_rate=0.002, beta_1=0.5)
  model.compile(loss=charbonnier_penalty, optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
charb=charbonnier_model(img_shape=(128,128,3))
charb.summary()

In [52]:
def train_charbonnier_model(input_batch, HR_batch):
  
  #extracting HR Image features through VGG
  HR_resized=resize_layer(HR_batch)
  ground_feat=vgg(HR_resized)

  loss, acc=charb.train_on_batch(input_batch, ground_feat)
  return loss, acc

In [53]:
def consistency_loss(y_true, y_pred):
  lambd=5
  return 5*charbonnier_penalty(y_true, y_pred)

def consistency_model(img_shape):
  inp=Input(shape=img_shape)
  Ibase=udsn(inp)
  ISR=eesn(Ibase)
  model=Model(inputs=inp, outputs=ISR)
  opt=Adam(learning_rate=0.002, beta_1=0.5)
  model.compile(loss=consistency_loss, optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
cons_model=consistency_model(img_shape=(128,128,3))
cons_model.summary()

In [55]:
def train_consistency_model(input_batch, HR_batch):
  loss, acc=cons_model.train_on_batch(input_batch, HR_batch)
  return loss, acc

In [59]:
def build_gan(img_shape):
  inp=Input(shape=img_shape)
  Ibase=udsn(inp)
  ISR=eesn(Ibase)
  out=disc(ISR)
  disc.trainable=False
  model=Model(inputs=inp, outputs=out)
  opt=Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
gan=build_gan(img_shape=(128,128,3))
gan.summary()

In [65]:
def generate_real_samples(n_batch):
  ix=np.random.randint(0, 1096, n_batch)
  HR_batch=src_list[ix]
  y_real=np.ones(shape=(n_batch,))
  return HR_batch, y_real

def generate_fake_samples(n_batch):
  ix=np.random.randint(0, 1096, n_batch)
  LR_batch=resized_list[ix]
  HR_batch_fake=eesn(udsn(LR_batch))
  y_fake=np.zeros(shape=(n_batch,))
  return HR_batch_fake, y_fake

In [63]:
def train(n_epochs=20, n_batch=2):
  half_batch=n_batch//2
  for i in range(n_batch):
    ix=np.random.randint(0, 1096, n_batch)
    LR_batch=resized_list[ix]
    HR_batch=src_list[ix]
    
    charb_loss, charb_acc=train_charbonnier_model(LR_batch, HR_batch)
    cons_loss, cons_acc=train_consistency_model(LR_batch, HR_batch)

    real_batch, y_real=generate_real_samples(half_batch)
    fake_batch, y_fake=generate_fake_samples(half_batch)

    disc_loss_real, disc_acc_real=disc.train_on_batch(real_batch, y_real)
    disc_loss_fake, disc_acc_fake=disc.train_on_batch(fake_batch, y_fake)

    false_negatives=np.ones(shape=(n_batch,))
    gan_loss, gan_acc= gan.train_on_batch(LR_batch, false_negatives)

    print('%d charb_loss[%.6f] cons_loss[%.6f] disc_loss_real[%.6f] disc_loss_fake[%.6f] gan_loss[%.6f]'%(i+1, charb_loss, cons_loss, disc_loss_real, 
                                                                                                          disc_loss_fake, gan_loss))

In [66]:
train(2, 2)

1 charb_loss[nan] cons_loss[nan] disc_loss_real[0.000000] disc_loss_fake[nan] gan_loss[nan]
2 charb_loss[nan] cons_loss[nan] disc_loss_real[nan] disc_loss_fake[nan] gan_loss[nan]
